In [1]:
from tools import load_data
import numpy as np
import glob
import os
import pandas as pd

2024-09-03 21:49:20.217005: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-09-03 21:49:20.217226: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-09-03 21:49:20.368116: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-09-03 21:49:20.598576: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-09-03 21:49:21.828697: W tensorflow/compiler/tf2

In [2]:
# QUIC Berkeley (quic text)
# QUIC Paris-Est Créteil(quic_pcaps)
data_dirs = ['/home/jony/Git/GAN-N-Net/datasets/mini_flowpic_flash_clean/',
            '/home/jony/Git/GAN-N-Net/datasets/mini_flowpic_quic_text/',
            '/home/jony/Git/GAN-N-Net/datasets/mini_flowpic_quic_pcaps/']

In [3]:
for data_dir in data_dirs:    
    files = glob.glob(data_dir + "*")
    CLASS_NUM = len(files)


    # GAN random input vector size
    latent_dim = 32*32*3
    data = []
    labels = []
    labelIndex = -1
    print(data_dir)
    for file in files:
        print(f"working on: {os.path.basename(file)}")
        if file.endswith('.npy'):
            labelIndex += 1
            dataForFile = load_data(file)
            print(dataForFile.shape)
            data.append(dataForFile)
            labelsForFile = np.ones(dataForFile.__len__()) * labelIndex
            labels.append(labelsForFile)


    data = np.vstack(data)
    labels = np.hstack(labels)
    
    print(data.shape)
    print(labels.shape)
    print(np.unique(labels,return_counts=True))
    print('\n\n\n')

/home/jony/Git/GAN-N-Net/datasets/mini_flowpic_flash_clean/
working on: csvs_instagram.npy
(46, 32, 32)
working on: csvs_facebook.npy
(320, 32, 32)
working on: csvs_tiktok.npy
(150, 32, 32)
working on: csvs_youtube.npy
(736, 32, 32)
(1252, 32, 32)
(1252,)
(array([0., 1., 2., 3.]), array([ 46, 320, 150, 736]))




/home/jony/Git/GAN-N-Net/datasets/mini_flowpic_quic_text/
working on: class_Google_Music.npy
(592, 32, 32)
working on: class_Google_Search.npy
(1915, 32, 32)
working on: class_Google_Doc.npy
(8003, 32, 32)
working on: class_Youtube.npy
(3521, 32, 32)
working on: quic_text.zip
working on: class_Google_Drive.npy
(2457, 32, 32)
(16488, 32, 32)
(16488,)
(array([0., 1., 2., 3., 4.]), array([ 592, 1915, 8003, 3521, 2457]))




/home/jony/Git/GAN-N-Net/datasets/mini_flowpic_quic_pcaps/
working on: first_Google_PlayMusic.npy
(490, 32, 32)
working on: first_YouTube.npy
(1152, 32, 32)
working on: first_GoogleHangout_Chat.npy
(1106, 32, 32)
working on: first_GoogleHangout_VoIP.npy
(6779,

In [4]:
data_dir = "/home/jony/Git/GAN-N-Net/datasets/not_prosseesd_yet/site5_1500_packets_al_to_share/csvs/"
# get list of all files in data_dir
files = glob.glob(data_dir + "*.csv")
files[:10]
df = pd.read_csv(files[0])
df['udps.OTT'].unique()
df.head()
print(len(df.iloc[0]["udps.packets_raw_sizes"]))
len(df.iloc[0]["udps.n_bytes"])

df.iloc[0]

7170


Unnamed: 0                                                                         0
bidirectional_first_seen_ms                                            1716475693796
new_application_types                                                          other
new_stnn                           [ 0.0000000e+00  2.6720000e+03  3.3725000e+02 ...
udps.packets_raw_sizes             [1454, 78, 1454, 78, 1454, 78, 1454, 100, 78, ...
udps.protocol_header_fields_enh    [[    0  1388     0   868     3    58]\n [    ...
udps.simple_tig_adj                [[0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...
udps.simple_tig_features           [[-1454.0], [78.0], [-1454.0], [78.0], [-1454....
application_category_name                                              SocialNetwork
udps.stnn_image                    [[0.0, 2672.0, 188.46875, 642.0958, 3.5761895,...
udps.OTT                                                                    OtherOTT
udps.flowType                                                    

In [5]:
# create an empty list to store the DataFrames
dfs = []

# loop through each CSV file and read it into a DataFrame
for file in files:
    df = pd.read_csv(file)
    df['filename'] = file.split('/')[-1]
    dfs.append(df)

# concatenate all DataFrames into a single one
big_df = pd.concat(dfs, ignore_index=True)

In [ ]:
big_df["len_packets_raw_sizes"] = big_df["udps.packets_raw_sizes"].apply(len)
big_df["len_n_bytes"] = big_df["udps.n_bytes"].apply(len)
big_df["len_packets_raw_sizes"].describe()
print(big_df["len_packets_raw_sizes"].describe())
print(big_df["len_n_bytes"].describe())

In [ ]:
# all can be found in EIMTC git repo (ask amit and chen)
# udps.protocol_header_fields_enh - nfstream plugin that contains:
    # Dirrection: 0 - source to destination, 1 - destination to source
    # payload_size: not include headers
    # delta_time: time from previus packet
    # scapy_tcp: I don't kknow what it is
    # len(scapy_tcp.options): I don't kknow what it is
    # scapy_ip.ttl: ttl value....


# type(big_df["udps.protocol_header_fields_enh"].iloc[0])
big_df["udps.protocol_header_fields_enh"].iloc[0]

# new_application_types, new_traffic_types

In [ ]:
# big_df['splited'] = big_df['udps.protocol_header_fields_enh'].apply(lambda x: [list(map(int, row.split())) for row in x.strip().split('\n')])
# big_df['splited'] = big_df['udps.protocol_header_fields_enh'].apply(lambda x: [list(map(int, row.strip('[]').split())) for row in x.strip().split('\n') if row.strip()])
big_df['splited'] = big_df['udps.protocol_header_fields_enh'].apply(lambda x: [list(map(int, row.strip().strip('[]').split())) for row in x.strip('[]').split('\n') if row.strip()])

In [ ]:
# lambda x: [list(map(int, row.split())) for row in x.strip().split('\n')]
l = df['udps.protocol_header_fields_enh'].iloc[0].strip('[]').split('\n')
# print(df['udps.protocol_header_fields_enh'].iloc[0])
list(map(int, l[0].strip('[]').split()))
l[1].strip().strip('[]').split()
# l[1]